# IFRS 9 ECL Risk Modeling System
## Google Colab Notebook

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jennibhroin/riskmodelling-tool/blob/main/IFRS9_ECL_Colab.ipynb)

This notebook allows you to run IFRS 9 Expected Credit Loss calculations directly in Google Colab.

**Features:**
- 🎯 No installation required
- 📊 Calculate ECL for credit portfolios
- 🌐 Multi-scenario analysis
- 📈 Probability-weighted ECL
- 💾 Upload your own portfolio data
- 📥 Download results

## 1. Setup: Clone Repository and Install Dependencies

Run this cell first to set up the environment.

In [ ]:
# Clone the repository
!git clone https://github.com/jennibhroin/riskmodelling-tool.git

# Change to project directory
import os
os.chdir('riskmodelling-tool')

# Install dependencies (suppress output for cleaner display)
!pip install -q -r requirements.txt

print("✅ Setup complete! Ready to calculate ECL.")

## 2. Basic ECL Calculation

Calculate ECL for the sample portfolio (25 loans).

In [ ]:
from data_management.portfolio_loader import PortfolioLoader
from core.ecl_engine import ECLCalculationEngine

# Load sample portfolio
print("Loading portfolio...")
items = PortfolioLoader.load_from_csv('examples/sample_portfolio.csv')
print(f"✅ Loaded {len(items)} portfolio items")

# Calculate ECL
print("\nCalculating ECL...")
engine = ECLCalculationEngine()
results = engine.calculate_portfolio_ecl(items)

# Display results
print("\n" + "="*60)
print("ECL CALCULATION RESULTS")
print("="*60)
print(f"Total ECL:          ${float(results.total_ecl):>15,.2f}")
print(f"Total Exposure:     ${float(results.total_exposure):>15,.2f}")
print(f"Coverage Ratio:     {results.coverage_ratio:>15.2%}")
print()
print(f"Total Items:        {results.total_items:>15,}")
print()
print("ECL by Stage:")
print("-"*60)
print(f"  Stage 1: {results.stage_1_count:2d} items, ${float(results.stage_1_ecl):>12,.2f} ({results.stage_1_coverage:>6.2%})")
print(f"  Stage 2: {results.stage_2_count:2d} items, ${float(results.stage_2_ecl):>12,.2f} ({results.stage_2_coverage:>6.2%})")
print(f"  Stage 3: {results.stage_3_count:2d} items, ${float(results.stage_3_ecl):>12,.2f} ({results.stage_3_coverage:>6.2%})")
print("="*60)

## 3. Multi-Scenario Analysis

Calculate ECL under multiple economic scenarios (Base, Optimistic, Pessimistic, Stress).

In [ ]:
from scenarios.scenario_manager import ScenarioManager
import pandas as pd

# Load scenarios
print("Loading economic scenarios...")
scenario_mgr = ScenarioManager()
scenario_mgr.load_from_config('config/scenarios_config.yaml')

print(f"\n✅ Loaded {len(scenario_mgr.list_scenarios())} scenarios:")
for scenario in scenario_mgr.list_scenarios():
    print(f"   - {scenario.name:<20} ({str(scenario.scenario_type):<12}) Prob: {scenario.probability:.1%}")

# Calculate ECL for each scenario
print("\nCalculating ECL under each scenario...\n")
scenario_results = {}
comparison_data = []

for scenario in scenario_mgr.list_scenarios():
    result = engine.calculate_portfolio_ecl(items, scenario=scenario)
    scenario_results[scenario.name] = result
    
    comparison_data.append({
        'Scenario': scenario.name,
        'Probability': f"{scenario.probability:.1%}",
        'Total ECL': float(result.total_ecl),
        'Coverage': f"{result.coverage_ratio:.2%}"
    })

# Display comparison table
print("="*80)
print("SCENARIO COMPARISON")
print("="*80)
df = pd.DataFrame(comparison_data)
print(df.to_string(index=False))

# Calculate probability-weighted ECL
weighted_ecl = scenario_mgr.calculate_weighted_ecl(scenario_results)
print("\n" + "-"*80)
print(f"Probability-Weighted ECL: ${float(weighted_ecl):,.2f}")
print("="*80)

# Show ECL range
ecl_values = [float(r.total_ecl) for r in scenario_results.values()]
print(f"\nECL Range: ${min(ecl_values):,.2f} - ${max(ecl_values):,.2f}")
print(f"Range Factor: {max(ecl_values)/min(ecl_values):.1f}x")

## 4. Visualize Results

Create charts to visualize ECL by stage and scenario.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create figure with subplots
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# 1. ECL by Stage (Pie Chart)
base_result = scenario_results['base']
stage_ecl = [
    float(base_result.stage_1_ecl),
    float(base_result.stage_2_ecl),
    float(base_result.stage_3_ecl)
]
stage_labels = ['Stage 1\n(12-month ECL)', 'Stage 2\n(Lifetime ECL)', 'Stage 3\n(Impaired)']
colors = ['#2ecc71', '#f39c12', '#e74c3c']

axes[0].pie(stage_ecl, labels=stage_labels, autopct='%1.1f%%', colors=colors, startangle=90)
axes[0].set_title('ECL Distribution by Stage\n(Base Scenario)', fontsize=14, fontweight='bold')

# 2. ECL by Scenario (Bar Chart)
scenarios = ['base', 'optimistic', 'pessimistic', 'stress_recession']
scenario_names = ['Base', 'Optimistic', 'Pessimistic', 'Recession\nStress']
scenario_ecl = [float(scenario_results[s].total_ecl) for s in scenarios if s in scenario_results]
scenario_colors = ['#3498db', '#2ecc71', '#e74c3c', '#8e44ad']

bars = axes[1].bar(scenario_names[:len(scenario_ecl)], scenario_ecl, color=scenario_colors[:len(scenario_ecl)])
axes[1].set_ylabel('Total ECL ($)', fontsize=12)
axes[1].set_title('ECL by Economic Scenario', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height,
                f'${height:,.0f}',
                ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

print("\n✅ Visualizations generated!")

## 5. Upload Your Own Portfolio

Upload a CSV file with your portfolio data and calculate ECL.

**Required CSV columns:**
- `item_id`, `borrower_id`, `origination_date`, `maturity_date`, `outstanding_amount`

**Optional columns:**
- `undrawn_commitment`, `credit_score`, `collateral_value`, `days_past_due`, `sector`, etc.

In [ ]:
from google.colab import files

# Upload file
print("Upload your portfolio CSV file:")
uploaded = files.upload()

# Get filename
filename = list(uploaded.keys())[0]
print(f"\n✅ File uploaded: {filename}")

# Load and calculate
try:
    print("\nLoading portfolio...")
    my_items = PortfolioLoader.load_from_csv(filename)
    print(f"✅ Loaded {len(my_items)} items")
    
    print("\nCalculating ECL...")
    my_results = engine.calculate_portfolio_ecl(my_items)
    
    # Display results
    print("\n" + "="*60)
    print("YOUR PORTFOLIO ECL RESULTS")
    print("="*60)
    print(f"Total ECL:          ${float(my_results.total_ecl):>15,.2f}")
    print(f"Total Exposure:     ${float(my_results.total_exposure):>15,.2f}")
    print(f"Coverage Ratio:     {my_results.coverage_ratio:>15.2%}")
    print(f"Total Items:        {my_results.total_items:>15,}")
    print("\nStage Distribution:")
    print(f"  Stage 1: {my_results.stage_1_count:3d} items ({my_results.stage_1_count/my_results.total_items*100:5.1f}%)")
    print(f"  Stage 2: {my_results.stage_2_count:3d} items ({my_results.stage_2_count/my_results.total_items*100:5.1f}%)")
    print(f"  Stage 3: {my_results.stage_3_count:3d} items ({my_results.stage_3_count/my_results.total_items*100:5.1f}%)")
    print("="*60)
    
except Exception as e:
    print(f"\n❌ Error: {str(e)}")
    print("\nPlease check your CSV format. See the sample portfolio for reference.")

## 6. Export Results

Export ECL results to Excel and download.

In [ ]:
from data_management.portfolio_exporter import PortfolioExporter
from google.colab import files

# Export base scenario results to Excel
output_file = 'ecl_results.xlsx'
print(f"Exporting results to {output_file}...")

PortfolioExporter.export_portfolio_ecl_to_excel(
    scenario_results['base'],
    output_file
)

print("✅ Export complete!")
print("\nDownloading file...")
files.download(output_file)

print("\n✅ Results downloaded! Check your downloads folder.")

## 7. Advanced: Custom Scenario

Create and run a custom economic scenario.

In [ ]:
from models.scenario_config import ScenarioConfig, MacroeconomicAdjustments
from models.enums import ScenarioType

# Create custom scenario
custom_macro = MacroeconomicAdjustments(
    gdp_growth=-1.5,           # GDP decline of 1.5 percentage points
    unemployment_rate=1.5,     # Unemployment increase of 1.5 pp
    credit_spreads=75,         # Credit spreads widen by 75 bps
    house_price_index=-5.0,    # House prices fall 5%
)

custom_scenario = ScenarioConfig(
    name='custom_mild_recession',
    scenario_type=ScenarioType.STRESS,
    probability=0.0,
    description='Custom mild recession scenario',
    macro_adjustments=custom_macro,
    pd_multiplier=1.15,        # 15% increase in PD
    lgd_multiplier=1.10,       # 10% increase in LGD
)

# Calculate ECL under custom scenario
print("Calculating ECL under custom scenario...\n")
custom_result = engine.calculate_portfolio_ecl(items, scenario=custom_scenario)

# Compare with base
base_ecl = float(scenario_results['base'].total_ecl)
custom_ecl = float(custom_result.total_ecl)
change_pct = (custom_ecl / base_ecl - 1) * 100

print("="*60)
print("CUSTOM SCENARIO RESULTS")
print("="*60)
print(f"Base Scenario ECL:     ${base_ecl:>15,.2f}")
print(f"Custom Scenario ECL:   ${custom_ecl:>15,.2f}")
print(f"Change:                {change_pct:>15.1f}%")
print("="*60)
print("\nMacroeconomic Adjustments:")
print(f"  GDP Growth:         {custom_macro.gdp_growth:>6.1f} pp")
print(f"  Unemployment:       {custom_macro.unemployment_rate:>6.1f} pp")
print(f"  Credit Spreads:     {custom_macro.credit_spreads:>6.0f} bps")
print(f"  House Prices:       {custom_macro.house_price_index:>6.1f}%")
print("="*60)

## Summary

You've successfully:
- ✅ Set up the IFRS 9 ECL system in Google Colab
- ✅ Calculated ECL for a sample portfolio
- ✅ Ran multi-scenario analysis
- ✅ Visualized results
- ✅ Learned how to upload your own data
- ✅ Exported results to Excel
- ✅ Created custom scenarios

### Next Steps:

- Review the [GitHub repository](https://github.com/jennibhroin/riskmodelling-tool) for more documentation
- Check the [Quick Start Guide](https://github.com/jennibhroin/riskmodelling-tool/blob/main/QUICKSTART.md)
- Explore configuration options in `config/` directory
- Review test cases for additional usage patterns

### Support:

- Repository: https://github.com/jennibhroin/riskmodelling-tool
- Documentation: See README.md in the repository

---

**Built with:** Python, pandas, numpy, IFRS 9 standards

**License:** MIT